## Подготовка данных для задания

In [5]:
import re
import pandas as pd
import pymorphy2

text_df = pd.read_csv("content_description.csv", sep='\t')
#text_df = pd.read_csv("./data/content_description.csv", sep='\t')
text_df.head(10)


,content,description
0,https://www.ivi.ru/watch/157318/description,"Лучший подарок, который только можно было прид..."
1,https://www.ivi.ru/watch/98336/description,Через какие трудности приходится проходить Сан...
2,https://www.ivi.ru/watch/183533/description,Миловидный Давид - позор для своего отца. Не в...
3,https://www.ivi.ru/watch/157319/description,Экранизация сатирического бестселлера Стивена ...
4,https://www.ivi.ru/watch/51342/description,«Леди удача» – авантюрная романтическая комеди...
5,https://www.ivi.ru/watch/183534/description,"На поминках Сюзанна узнает, что Джефф изменял ..."
6,https://www.ivi.ru/watch/112509/description,"Культовый японский фильм ужасов, заставляющий ..."
7,https://www.ivi.ru/watch/157320/description,Двое выпускников калифорнийской школы бизнеса ...


In [7]:

# разбиваем текст на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)

# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

print(corpus[0])

['лучший', 'подарок', 'который', 'только', 'можно', 'было', 'придумать', 'для', 'всех', 'поклонников', 'культового', 'сериала', 'рождественский', 'спецвыпуск', 'шерлока', 'в', 'котором', 'абсолютно', 'все', 'будет', 'особенным', 'и', 'старинные', 'костюмы', 'и', 'дух', 'викторианской', 'англии', 'и', 'конечно', 'же', 'покрытая', 'мраком', 'и', 'завесой', 'мистики', 'тайна', 'которую', 'предстоит', 'разгадать', 'гениальному', 'сыщику', 'кроме', 'того', 'создатели', 'сериала', 'обещали', 'оставить', 'для', 'самых', 'внимательных', 'поклонников', 'шерлока', 'подсказки', 'связанные', 'с', 'содержанием', 'четвертого', 'сезона', 'сериала', 'впервые', 'герои', 'современного', 'сериала', 'предстанут', 'в', 'своих', 'книжных', 'образах', 'холмс', 'с', 'трубкой', 'и', 'в', 'охотничьей', 'шляпе', 'а', 'ватсон', 'с', 'усами', 'и', 'в', 'котелке', 'действие', 'перенесено', 'в', '1895', 'год', 'в', 'центре', 'сюжета', 'загадочная', 'история', 'о', 'призраке', 'невесты', 'который', 'наводит', 'ужас',

In [11]:

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

print(normalized_corpus[0])

['хороший', 'подарок', 'который', 'только', 'можно', 'быть', 'придумать', 'для', 'весь', 'поклонник', 'культовый', 'сериал', 'рождественский', 'спецвыпуск', 'шерлок', 'в', 'который', 'абсолютно', 'всё', 'быть', 'особенный', 'и', 'старинный', 'костюм', 'и', 'дух', 'викторианский', 'англия', 'и', 'конечно', 'же', 'покрыть', 'мрак', 'и', 'завеса', 'мистика', 'тайна', 'который', 'предстоять', 'разгадать', 'гениальный', 'сыщик', 'кроме', 'тот', 'создатель', 'сериал', 'обещать', 'оставить', 'для', 'самый', 'внимательный', 'поклонник', 'шерлок', 'подсказка', 'связанный', 'с', 'содержание', 'четвёртый', 'сезон', 'сериал', 'впервые', 'герой', 'современный', 'сериал', 'предстать', 'в', 'свой', 'книжный', 'образ', 'холмс', 'с', 'трубка', 'и', 'в', 'охотничий', 'шляпа', 'а', 'ватсон', 'с', 'ус', 'и', 'в', 'котелок', 'действие', 'перенести', 'в', '1895', 'год', 'в', 'центр', 'сюжет', 'загадочный', 'история', 'о', 'призрак', 'невеста', 'который', 'наводить', 'ужас', 'на', 'весь', 'город', 'к', 'шерл

In [15]:
# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
29,0,и,7
37,0,который,5
111,0,шерлок,4
77,0,с,4
84,0,сериал,4
75,0,риколетти,3
31,0,история,2
21,0,для,2
49,0,о,2


# 12.3  Домашняя работа

12.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [157]:
def grammemme(word: str) -> str:
    p = morph.parse(word)[0]
    word_grammemme = p.tag.POS
    return str(word_grammemme)

doc_frequency = tokens_df.groupby('word', as_index=False).agg({'doc_id': pd.Series.nunique, 'dummy': np.sum}).sort_values('doc_id', ascending=False)

grammeme_to_exclude = ['PREP', 'None', 'CONJ', 'PRCL', 'INTJ']
unique_doc_number = len(word_count_df['doc_id'].unique())
    
doc_frequency['grammeme'] = doc_frequency['word'].apply(grammemme)

In [158]:
doc_frequency_fin = doc_frequency.copy()
doc_frequency_fin = doc_frequency_fin.drop(doc_frequency_fin[doc_frequency_fin.grammeme.isin(grammeme_to_exclude)].index)
doc_frequency_fin['freg'] = doc_frequency_fin['doc_id'] / n * 100


In [159]:
doc_frequency_fin.head(10)

,word,doc_id,dummy,grammeme,freg
323,он,6,18,NPRO,75.0
480,свой,6,9,ADJF,75.0
326,онлайн,6,6,ADVB,75.0
90,год,5,6,NOUN,62.5
498,смотреть,5,5,INFN,62.5
470,сам,4,8,ADJF,50.0
483,себя,4,4,NPRO,50.0
565,тот,4,4,ADJF,50.0
471,самый,4,6,ADJF,50.0
218,который,4,8,ADJF,50.0


12.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [483]:
# запишите необходимый текст description 
raw_text = text_df.description.iloc[3]
raw_text



'Экранизация сатирического бестселлера Стивена Фрая, мастерски воплощенная британским режиссером и продюсером Джоном Дженксом.   У Теда Уоллеса было славное прошлое, и вот-вот его книги должны были пополнить стройный ряд томов британских классиков. Но его жуткий характер вкупе с алкоголизмом и мизантропией в запущенной стадии испортили все. Теперь Тед зарабатывает на жизнь тем, что поносит в своих статьях спектакли, которые вообще-то не стоят его внимания. И продолжает пить. Вскоре Тед лишается и этого: его бесконечное ворчание достало всю редакцию.  Готовясь к голодной и бесславной смерти, внезапно Уоллес получает выгодное предложение от своей крестницы Джейн. Девушка недавно излечилась от болезни благодаря чуду и теперь предлагает Теду погостить в аристократическом доме, чтобы осветить происходящие здесь чудеса исцеления.  Старый скептик соглашается из корыстных побуждений и даже представить не может, сколько сюрпризов ему уготовано.  Смотреть онлайн эту комедию о том, что даже самые

In [522]:
# запишите необходимое регулярное выражение в переменную reg_expr 
reg_expr = r'[А-Я][а-я]+\s+[А-Я][а-я]+(?:\s+[А-Я][а-я]+)?'


In [523]:
# скомпилированное выражение добавьте в обьёект reg_expr_compiled
reg_expr_compiled = re.compile(reg_expr)


In [532]:

# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    for each in g.split():
        if grammemme(each) != 'NOUN':
            break
    else:
        print(g)


Стивена Фрая
Джоном Дженксом
Теда Уоллеса


10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`, сохранить в переменнуб `genres_df`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [680]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}


# -- ВАШ КОД ЗДЕСЬ --

In [681]:
genres_df = pd.DataFrame.from_dict(genre_dict, orient='index', columns=['w1', 'w2', 'w3'])
genres_df = genres_df.reset_index().rename(columns={'index': 'genre'})
genres_df


,genre,w1,w2,w3
0,комедия,сатирический,авантюрный,забавный
1,мелодрама,выбор,позор,None
2,сказка,приключения,милый,семейный
3,детектив,тайна,разгадать,загадочный
4,триллер,ужас,зловещий,нерв


In [702]:
col1, col2 = itertools.tee(['w1', 'w2', 'w3'])
new1 = pd.merge(word_count_df[['doc_id', 'word']], genres_df[['w1', 'genre']], how='inner', left_on='word', right_on='w1').rename(columns={'genre': 'genre1'})[['doc_id', 'genre1' ]]
new2 = pd.merge(word_count_df[['doc_id', 'word']], genres_df[['w2', 'genre']], how='inner', left_on='word', right_on='w2').rename(columns={'genre': 'genre2'})[['doc_id', 'genre2' ]]
new3 = pd.merge(word_count_df[['doc_id', 'word']], genres_df[['w3', 'genre']], how='inner', left_on='word', right_on='w3').rename(columns={'genre': 'genre3'})[['doc_id', 'genre3' ]]


In [722]:

genres_df_fin = pd.merge(pd.merge(new1, new2, on='doc_id', how='outer'), new2, on='doc_id', how='outer').fillna('')
genres_df_fin['genre_fin'] = genres_df_fin.apply(lambda genres_df_fin: ' '.join(list(set([str(genres_df_fin.genre1), str(genres_df_fin.genre2_x), str(genres_df_fin.genre2_y)]))), axis=1)
genres_df_fin[['doc_id', 'genre_fin']].rename(columns={'genre_fin': 'genre'})



,doc_id,genre
0,0,детектив
1,0,детектив триллер
2,6,триллер
3,2,мелодрама
4,3,комедия
5,1,сказка
6,4,комедия
